In [1]:
def vite_sync ():
    !cd ../vite && ls && yarn build
    !cd ../wp && rm -rf dist 
    !cp -r ../vite/dist ../wp/dist
    !cd ../wp && wp-env start
    # print url http://localhost:8888/
    print ( "http://localhost:8888/" )


In [ ]:
def slidev_pdf (src):
    # get timestamp in format ymd-His
    import datetime
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d-%H%M%S")
    #remove extension from src

    basesrc = src.split('.')[0]
    target = './my-pdfs/' + basesrc + '-' + timestamp + '.pdf'
    print(target)

    !cd ../slidev && slidev export $src --dark --output $target

    # return the path to the pdf
    return target



In [ ]:
def slidev_pdf2png (pdf_export):
    # get basename
    import os
    pdf_export = os.path.basename(pdf_export)
    # create subfolder with basename without extension
    png_folder = os.path.splitext(pdf_export)[0]
    !mkdir -p ../slidev/my-pdfs/$png_folder
    # replace pdf extension by png
    png_export = pdf_export.replace('.pdf', '/slide.png')

    # convert pdf to png
    print (pdf_export)
    !cd ../slidev/my-pdfs && convert -density 150 -colorspace sRGB -quality 100 -resize 1920x1920 $pdf_export $png_export
 
    # show files in folder
    !ls -lhtr ../slidev/my-pdfs/$png_folder

    # convert png files to mp4 video
    # !cd ../slidev/my-pdfs/$png_folder && ffmpeg -framerate 1/5 -i ./slide-%d.png -c:v libx264 -r 30 -pix_fmt yuv420p ../$png_folder.mp4
    return png_folder


In [ ]:
def slidev_png2mp4 (png_folder, png_file='slide-%3d.png'):
    print(png_folder)

    # get current dir
    import os
    curdir = os.getcwd()
    print(curdir)

    # change dir to ../slidev/my-pdfs/$png_folder
    os.chdir('../slidev/my-pdfs/' + png_folder)
    !pwd
    # convert png files to mp4 video
    video_file = '../' + png_folder + '.mp4'

    !ffmpeg -framerate 1/5 -i $png_file -c:v libx264 -r 30 -pix_fmt yuv420p $video_file

    # change dir to curdir
    os.chdir(curdir)
    !pwd

    print(video_file)
    return video_file
    

In [ ]:
def slidev_png (src):
    # get timestamp in format ymd-His
    import datetime
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d-%H%M%S")
    #remove extension from src
    basesrc = src.split('.')[0]
    png_folder = basesrc + '-' + timestamp
    target = './my-pdfs/' + png_folder
    png_target = '../slidev/my-pdfs/' + png_folder

    print(target)

    !cd ../slidev && slidev export --dark --format png --timeout 600000 --output $target $src

    # rename all png files with prefix slide-
    list_png = !ls $png_target
    print(list_png)
    for png in list_png:
        # rename png file with prefix slide-
        !mv $png_target/$png $png_target/slide-$png
    
    return png_folder



In [ ]:
def slidev_build_video (src):
    # call function slidev_png
    png_folder = slidev_png(src)
    video_file = slidev_png2mp4(png_folder)
    # get basename
    import os
    video_file = os.path.basename(video_file)
    return video_file

